In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers import GRU, LSTM
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
import io
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

def load_data_csv_to_dataframe(file_path):
    data = pd.read_csv(file_path)
    return data

def save_data_dataframe_to_csv(data,filename = None):
    if(filename):
        data.to_csv(filename,index=False)

    else:
        data.to_csv(index=False)

def evaluate():
    pass

def preprocess_file(filename):
    data = load_data_csv_to_dataframe(filename)
    for index, row in data.iterrows():
        text = row['comment_text']
        text = remove_puntucation(text)
        text = tokennize(text)
        text = fileter_stopwords(text)
        row['comment_text'] = text
        data.at[index,'comment_text'] = text

    return data


def remove_puntucation(sentence):
    sentence = "".join([char for char in sentence if char not in string.punctuation])
    return sentence


def tokennize(sentece):

    words = word_tokenize(sentece)
    return words

def fileter_stopwords(words):
    stop_words = stopwords.words('english')
    words = [word for word in words if word not in stop_words]
    return words

def stemming(words):
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    return words




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
def embedding(train,test):
    tokenizer = Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(train)
    t=len(tokenizer.word_index)+1
    trainsequences = tokenizer.texts_to_sequences(train)
    traindata = pad_sequences(trainsequences, maxlen=100)
    testsequences = tokenizer.texts_to_sequences(test)
    testdata = pad_sequences(testsequences, maxlen=100)
    return traindata, testdata,t

# Prepare train and test data
# df= pd.read_csv(io.BytesIO(uploaded['train.csv']),encoding='latin-1') 
# df1 = df.head(10000)
df = preprocess_file(io.BytesIO(uploaded['train.csv']))
X=df['comment_text']
classes=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y=df[classes].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

xtrain,xtest,vocab_size=embedding(X_train,X_test)

# Build the CNN model
def buildModel(xtrain,ytrain):
    batch_size=1000
    epochs=5
    model= Sequential()
    model.add(Embedding(20000,32,input_length=100))
    model.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.35))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(GRU(50,return_sequences=True))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epochs)
    model.save("toxic.h5")
    # figure out how to save the model
    return model

In [35]:
print('--- starting to build the model ---')
CNN_model = buildModel(xtrain,y_train)
print('--- Finish the building ---')

--- starting to build the model ---
Epoch 1/5
128/128 [==============================] - 32s 231ms/step - loss: 0.3077 - accuracy: 0.3378
Epoch 2/5
128/128 [==============================] - 30s 231ms/step - loss: 0.1371 - accuracy: 0.7124
Epoch 3/5
128/128 [==============================] - 29s 230ms/step - loss: 0.1138 - accuracy: 0.8327
Epoch 4/5
128/128 [==============================] - 29s 230ms/step - loss: 0.0916 - accuracy: 0.9216
Epoch 5/5
128/128 [==============================] - 29s 229ms/step - loss: 0.0842 - accuracy: 0.9381
--- Finish the building ---


In [7]:
y_pred = np.around(CNN_model.predict(xtest))

In [15]:
CNN_acc = metrics.accuracy_score(y_test,y_pred)
print('The accuracy for CNN model is', CNN_acc)

The accuracy for CNN model is 0.903368322105593


In [13]:
CNN_cm = metrics.multilabel_confusion_matrix(y_test, y_pred)
print('The confusion matrix for each label in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"] is:')
print(CNN_cm)

The confusion matrix for each label in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"] is:
[[[28541   271]
  [ 2003  1100]]

 [[31596    16]
  [  272    31]]

 [[30005   223]
  [ 1051   636]]

 [[31814     0]
  [  101     0]]

 [[30086   255]
  [ 1022   552]]

 [[31629     0]
  [  286     0]]]


In [16]:
print(metrics.classification_report(y_test, y_pred, target_names=classes,zero_division = 1))

               precision    recall  f1-score   support

        toxic       0.80      0.35      0.49      3103
 severe_toxic       0.66      0.10      0.18       303
      obscene       0.74      0.38      0.50      1687
       threat       1.00      0.00      0.00       101
       insult       0.68      0.35      0.46      1574
identity_hate       1.00      0.00      0.00       286

    micro avg       0.75      0.33      0.46      7054
    macro avg       0.81      0.20      0.27      7054
 weighted avg       0.77      0.33      0.45      7054
  samples avg       0.99      0.93      0.92      7054

